Urban Data Science & Smart Cities <br>
URSP688Y Spring 2025<br>
Instructor: Chester Harvey <br>
Urban Studies & Planning <br>
National Center for Smart Growth <br>
University of Maryland

# Demo 4 - Loading and Joining Data From Files

- Table gymnastics (a.k.a., data wrangling)
    - Load data from a CSV
    - Practice exploring the data
    - Convert dates stored as strings to `datetime` data types
    - Drop duplicate rows
    - Count rows withing groups
    - Concatenate tables
    - Join columns from another table
- Debugging

In [1]:
# Import packages
import pandas as pd

## Loading Data from a File

Let's get our hands on some real-world data by loading a table from a file.

Let's load data from the [Maryland Eviction Case Database](https://opendata.maryland.gov/Housing/District-Court-of-Maryland-Eviction-Case-Data/mvqb-b4hf/data).

A CSV file that is stored in the same directory as our notebook can be opened by entering just the file name as an argument to `pd.read_csv`.

In [2]:
df = pd.read_csv('District_Court_of_Maryland_Eviction_Case_Data_MG_PG.csv')

Let's practice navigating and doing some analysis with our DataFrame.

Preview the dataframe

In [3]:
df.head(2)

,Unnamed: 0,Event Date,Event Type,Event Comment,County,Location,Tenant City,Tenant State,Tenant ZIP Code,Case Type,Case Number,Evicted Date,Event Year,Eviction Year
0,0,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20910.0,Failure to Pay Rent,D-061-LT-22-004107,12/08/2022,2023.0,2022.0
1,1,01/03/2023,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20910.0,Failure to Pay Rent,D-061-LT-22-000755,12/08/2022,2023.0,2022.0


How many rows does it have?

In [4]:
len(df)

96077

What columns does it have?

In [5]:
df.columns.tolist()

['Unnamed: 0',
 'Event Date',
 'Event Type',
 'Event Comment',
 'County',
 'Location',
 'Tenant City',
 'Tenant State',
 'Tenant ZIP Code',
 'Case Type',
 'Case Number',
 'Evicted Date',
 'Event Year',
 'Eviction Year']

Which counties are represented?

In [6]:
df['County'].value_counts()

County
Prince George's    67003
Montgomery         29074
Name: count, dtype: int64

What is the earlist date?

Is this true?

In [7]:
df.sort_values('Event Date', ascending=True).head(2)

,Unnamed: 0,Event Date,Event Type,Event Comment,County,Location,Tenant City,Tenant State,Tenant ZIP Code,Case Type,Case Number,Evicted Date,Event Year,Eviction Year
37027,37027,01/01/2024,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,SILVER SPRING,MD,20910.0,Failure to Pay Rent,D-061-LT-23-018218,NaN,2024.0,NaN
37051,37051,01/01/2024,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,SILVER SPRING,MD,20910.0,Failure to Pay Rent,D-061-LT-23-015290,NaN,2024.0,NaN


Convert the event date column to a `datetime` data type

In [8]:
df['Event Date'] = pd.to_datetime(df['Event Date'])

In [9]:
df.sort_values('Event Date', ascending=True).head(2)

,Unnamed: 0,Event Date,Event Type,Event Comment,County,Location,Tenant City,Tenant State,Tenant ZIP Code,Case Type,Case Number,Evicted Date,Event Year,Eviction Year
0,0,2023-01-03,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20910.0,Failure to Pay Rent,D-061-LT-22-004107,12/08/2022,2023.0,2022.0
20,20,2023-01-03,Warrant of Restitution - Return of Service - E...,NaN,Montgomery,Rockville,Silver Spring,MD,20901.0,Tenant Holding Over,D-06-CV-22-010697,09/26/2022,2023.0,2022.0


How many unique cases are there?

In [10]:
df_dedup = df.sort_values('Event Date', ascending=True).drop_duplicates(subset='Case Number', keep='first')

In [11]:
len(df_dedup)

67230

How many unique cases per zip code?

In [12]:
# Count cases within each zip code
cases_per_zip = df_dedup.groupby('Tenant ZIP Code')['Case Number'].count().sort_values(ascending=False)

cases_per_zip

Tenant ZIP Code
20746.0    5490
20770.0    5013
20747.0    4495
20748.0    4001
20708.0    3710
           ... 
20474.0       1
20447.0       1
19802.0       1
20151.0       1
55403.0       1
Name: Case Number, Length: 128, dtype: int64

In [13]:
# Convert results to a dataframe with a column for zip and a column for cases
cases_per_zip = pd.DataFrame(cases_per_zip).reset_index()
cases_per_zip = cases_per_zip.rename(columns={'Tenant ZIP Code':'case_zip','Case Number':'case_count'})

cases_per_zip.head()

,case_zip,case_count
0,20746.0,5490
1,20770.0,5013
2,20747.0,4495
3,20748.0,4001
4,20708.0,3710


Which zip codes have the most unique cases per person?

Let's join data from [CensusReporter](https://censusreporter.org/).

### Combining/Merging/Joining Tables

Combining information from multiple tables into a single table is one of the most useful data wrangling operations.

There are lots of different ways to join tables, but two basic types are:

1. Joining column with a shared key, which outputs a table that is wider than either input.
2. Concatenating rows with shared column names, which outputs a table that is longer than either input.

#### Joining columns based on a key

![joining columns with a shared key](https://rforhr.com/horizontal_join.png)


#### Concatenating rows with the same column names
![joining rows with shared column names](https://rforhr.com/vertical_join.png)

First, let's concatenate census tables for Montgomery and Prince George's county to make a single table with populations for each zip code.

Then, we'll merge counts of eviction cases onto each zip code.

Finally, we'll calcuate the number of eviction cases per capita.

In [14]:
# Load census reporter data, ignoring the row with data for the whole county (first row under the header)
def load_census_reporter_csv(path, skiprows=[1]):
    return pd.read_csv(path, skiprows=skiprows)

df_pop_mg = load_census_reporter_csv('acs2023_5yr_B01003_mg.csv')
df_pop_pg = load_census_reporter_csv('acs2023_5yr_B01003_pg.csv') 

In [15]:
# Combine into a single dataframe
df_pop = pd.concat([df_pop_mg, df_pop_pg], axis=0)

In [16]:
# Rename columns with readable names
df_pop = df_pop.rename(columns={'name':'census_zip', 'B01003001':'population', 'B01003001, Error':'population_error'})

In [17]:
df_pop.head()

,geoid,census_zip,population,population_error
0,86000US20705,20705,28119,1864
1,86000US20707,20707,36549,1231
2,86000US20777,20777,3046,543
3,86000US20812,20812,314,117
4,86000US20814,20814,30822,1739


In [18]:
df_pop.census_zip.dtype

dtype('int64')

In [19]:
# Merge the case counts onto the zip codes
cases_per_zip.head()

,case_zip,case_count
0,20746.0,5490
1,20770.0,5013
2,20747.0,4495
3,20748.0,4001
4,20708.0,3710


In [20]:
# Make sure zip codes are stored as strings in both dataframes
cases_per_zip['case_zip'] = cases_per_zip['case_zip'].astype('int64').astype('string')
df_pop['census_zip'] = df_pop['census_zip'].astype('string')

In [21]:
df_pop = df_pop.merge(cases_per_zip, left_on='census_zip', right_on='case_zip', how='left')

In [22]:
df_pop.head()

,geoid,census_zip,population,population_error,case_zip,case_count
0,86000US20705,20705,28119,1864,20705,1002.0
1,86000US20707,20707,36549,1231,20707,1101.0
2,86000US20777,20777,3046,543,<NA>,NaN
3,86000US20812,20812,314,117,<NA>,NaN
4,86000US20814,20814,30822,1739,20814,185.0


In [23]:
# Cleanup
df_pop['case_count'] = df_pop['case_count'].fillna(0)
df_pop = df_pop.drop(columns=['population_error','case_zip'])

In [24]:
df_pop.head()

,geoid,census_zip,population,case_count
0,86000US20705,20705,28119,1002.0
1,86000US20707,20707,36549,1101.0
2,86000US20777,20777,3046,0.0
3,86000US20812,20812,314,0.0
4,86000US20814,20814,30822,185.0


In [25]:
df_pop['cases_per_pop'] = df_pop['case_count'] / df_pop['population']

In [26]:
df_pop.sort_values('cases_per_pop', ascending=False)

,geoid,census_zip,population,case_count,cases_per_pop
76,86000US20746,20746,27494,5490.0,0.199680
81,86000US20770,20770,28181,5013.0,0.177886
61,86000US20708,20708,26599,3710.0,0.139479
75,86000US20745,20745,29518,3648.0,0.123586
77,86000US20747,20747,37924,4495.0,0.118527
...,...,...,...,...,...
2,86000US20777,20777,3046,0.0,0.000000
12,86000US20838,20838,409,0.0,0.000000
38,86000US20892,20892,0,0.0,NaN
39,86000US20894,20894,0,0.0,NaN


## Errors and debugging

Errors are frustrating and inevitable. Even professional programmers probably spend most of their time debugging.

Luckily, there are good tools and techniques for making debugging a little easier.

Despite these, you will probably nearly tear your hair out with some frequency, especially as a beginner. It will get better with time.

There are two types of errors in programming: logic and syntax. They both result in your program not achieving its goal, but the first may not be as easily detectable because the code may still run.

### Logic errors
These are issues with how you have approached or executed your problem. If your code runs but produces nonsensical results, there is probably a logic error. However, your erroneous code might also produce logical but *wrong* results; you might never notice until the problem has rippled downstream. It's best to address this proactively by planning your code well so it's less likely to be illogical, and writing readable code that can be easily reviewed.

Here's a logic error. Can you find it? (Hint: the issue is syntactical, but it's still a logic error because the code works without throwing an error.)

In [27]:
def check_adult(age, cutoff=18):
    if age > cutoff:
        adult = False
    else:
        adult = True
    return adult

check_adult(20)

False

### Syntax errors
These are more obvious because your code will simply fail. There are lots of tools for figuring out where and why.

Error messages are usually the starting place for debugging a syntax error.

In [28]:
def check_adult(age, cutoff=18):
    if age < cutoff:
        adult = False
    else:
        adult = True
    return adult

check_adult('20')

TypeError: '<' not supported between instances of 'str' and 'int'

The error message tells us where the problem is located.

Sometimes, it can be helpful to turn on line numbers.
- In Colab: `Tools -> Settings -> Editor -> Show line numbers`
- In JupyterLab: `View -> Show Line Numbers`

The `ValueError` tells us that the issue is related to the value of a variable on this line, but it's still pretty vague.

Time to start [Googling](https://www.google.com/).


### Debugging
We can also use an "interactive debugger" to help diagnose our problem by stepping through the code one line at a time.

The debugger provides tools for setting "breakpoints" where the code will stop running temporarily, a table that shows the values of variables at that time, and buttons to start, stop, and step through the code.

https://jupyterlab.readthedocs.io/en/stable/user/debugger.html

In [29]:
def check_adult(age, cutoff=18):
    if age < cutoff:
        adult = False
    else:
        adult = True
    return adult

check_adult(20)

True

## Style guidelines for Python
- At the very least, do things consistently
- One statement per line
- Try to limit line length to 72 characters
- Use four spaces to indent
- Put spaces around operators (e.g., `1 + 1` or `day = 'Monday'`) (except in keyword function arguments)
- Use blank lines intentionally and consistently
- Use meaningful names
- Name variables and functions with `lowercase_underscores`
- Constants are often named in `ALL_CAPS_WITH_UNDERSCORES` (e.g., `C = 2.99792458e+8`)
- Name custom classes with `CapWords`
- In general, avoid spaces in folder and filenames used for programming

See [Code Readability](https://github.com/ncsg/ursp688y_sp2024/blob/main/README.md#code-readability) on the syllabus. [CS61A](https://cs61a.org/articles/composition/) has an excellent composition guide. [PEP 8](https://peps.python.org/pep-0008/) is a standard Python style guide. [Google](https://google.github.io/styleguide/pyguide.html) publishes their internal Python style guide.